In [1]:
import arbor
import re

In [2]:
import pyNN.arbor as sim

In [3]:
na_ion={"reversal_potential": 50.0}
k_ion={"reversal_potential": -77.0}

In [4]:
decor = arbor.decor()

In [5]:
ionic_species={'na_ion': sim.NaIon, 'k_ion': sim.KIon}
other_parameters = {"na_ion": {"reversal_potential": 50.0}, "k_ion": {"reversal_potential": -77.0}}

In [18]:
ionic_species["na_ion"].translate(param = {"na_ion": {"reversal_potential": 50.0}})

TypeError: translate() got an unexpected keyword argument 'param'

In [23]:
ionic_species["na_ion"].translations

{'reversal_potential': {'translated_name': 'rev_pot',
  'forward_transform': 'reversal_potential',
  'reverse_transform': 'rev_pot'},
 'internal_concentration': {'translated_name': 'int_con',
  'forward_transform': 'internal_concentration',
  'reverse_transform': 'int_con'},
 'external_concentration': {'translated_name': 'ext_con',
  'forward_transform': 'external_concentration',
  'reverse_transform': 'ext_con'}}

In [24]:
ionic_species["na_ion"].translations["reversal_potential"]

{'translated_name': 'rev_pot',
 'forward_transform': 'reversal_potential',
 'reverse_transform': 'rev_pot'}

In [25]:
ionic_species["na_ion"].translations["reversal_potential"]["translated_name"]

'rev_pot'

In [29]:
ionic_species["na_ion"].model

'na'

In [8]:
other_parameters = {"ionic_species": {"na": {"reversal_potential": 50.0}, "k": {"reversal_potential": -77.0}}}

In [12]:
def _decorate_ionic_species(other_parameters):
    """
    Cal this as self._decorate_ionic_species(other_parameters) to update self._decor
    """
    translated_keys = {"external_concentration": "ext_con",
                       "internal_concentration": "int_con",
                       "ion_name": "ion_name",
                       "reversal_potential": "rev_pot"}
    for ion_name, ion_sp in other_parameters["ionic_species"].items():
        dict_input = _get_dict_for_ion(translated_keys, ion_sp)
        print(dict_input)
        #decor.set_ion(getattr(ion_sp, "ion_name"), **dict_input)

def _get_dict_for_ion(translated_keys, the_ionic_species):
    keys_in_the_ionic_species = [a for a in dir(the_ionic_species) if not a.startswith('__')]
    dict_for_ion = {}
    for key in keys_in_the_ionic_species:
        if not getattr(the_ionic_species, key):
            pass  # empty so do nothing
        else:
            if key == "ion_name":
                dict_for_ion.update({"method": "nernst/" + getattr(the_ionic_species, key)})
            else:
                dict_for_ion.update({translated_keys[key]: getattr(the_ionic_species, key)})
    return dict_for_ion

In [13]:
_decorate_ionic_species(other_parameters)

KeyError: 'clear'

In [15]:
#decor.set_ion('ca', method=mech('nernst/x=ca'))
#decor.set_ion('na', int_con=5.0, rev_pot=70, method=None)

In [16]:
dicts = {"int_con": 5.0, "rev_pot":70, "method": None}
decor.set_ion("na", **dicts)

In [17]:
ionic_species={'na_ion': sim.NaIon}

In [26]:
s = "na_ion"

In [27]:
s.split("_")

['na', 'ion']

In [28]:
"na_ion".split("_")[0]

'na'

In [2]:
arbor.mechanism("hh/e=-70")

<arbor.mechanism: name 'hh/e=-70', parameters {}>

In [3]:
m1 = arbor.mechanism("hh")

In [4]:
m1.name

'hh'

In [5]:
m1.values

{}

In [6]:
help(m1)

Help on mechanism in module arbor._arbor object:

class mechanism(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      mechanism
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: arbor._arbor.mechanism, name: str) -> None
 |      
 |      The name of the mechanism
 |      
 |      2. __init__(self: arbor._arbor.mechanism, name: str, params: Dict[str, float]) -> None
 |      
 |      Example usage setting parameters:
 |        m = arbor.mechanism('expsyn', {'tau': 1.4})
 |      will create parameters for the 'expsyn' mechanism, with the provided value
 |      for 'tau' overrides the default. If a parameter is not set, the default
 |      (as defined in NMODL) is used.
 |      
 |      Example overriding a global parameter:
 |        m = arbor.mechanism('nernst/R=8.3145,F=96485')
 |  
 |  __repr__(...)
 

In [2]:
d = {}

In [5]:
if not d:
    print("empty")

empty


In [1]:
import arbor
import re
import neuroml

In [2]:
import pyNN.arbor as sim
sim.setup(timestep=0.025)

0

In [69]:
from pyNN.morphology import NeuroMLMorphology
from neuroml import Morphology, Segment, Point3DWithDiam as P
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
nu_morph = NeuroMLMorphology(Morphology(segments=(soma, dend)))

In [39]:
cell_class = sim.MultiCompartmentNeuron
cell_class.label = "ExampleMultiCompartmentNeuron"
cell_class.ion_channels = {'pas': sim.PassiveLeak, 'na': sim.NaChannel, 'kdr': sim.KdrChannel}

In [40]:
nu = sim.MultiCompartmentNeuron.setup(
				label="ExampleMultiCompartmentNeuron",
				ion_channels={'pas': sim.PassiveLeak,
							  'na': sim.NaChannel,
							  'kdr': sim.KdrChannel},
				ionic_species={'na_ion': sim.NaIon,
							   'k_ion': sim.KIon})

In [41]:
nu_morph

In [42]:
for indx, nml_seg in enumerate(nu_morph.segments):
    print(nml_seg, nml_seg.name, type(nml_seg))

<Segment|0|soma> soma <class 'neuroml.nml.nml.Segment'>
<Segment|1|dendrites> dendrites <class 'neuroml.nml.nml.Segment'>


In [43]:
for indx, nml_seg in enumerate(nu_morph.segments):
    if isinstance( nml_seg, neuroml.nml.nml.Segment):
        print("is a segment")

is a segment
is a segment


In [44]:
nu_morph.soma_index

AttributeError: 'NeuroMLMorphology' object has no attribute '_soma_index'

In [45]:
nu_morph.segments

(<Segment|0|soma>, <Segment|1|dendrites>)

In [46]:
nu_morph.soma_index

AttributeError: 'NeuroMLMorphology' object has no attribute '_soma_index'

In [47]:
nu_morph.segments

(<Segment|0|soma>, <Segment|1|dendrites>)

In [48]:
from pyNN.morphology import NeuroMLMorphology, load_morphology, uniform, random_section, dendrites, apical_dendrites, by_distance

In [49]:
na={"conductance_density": uniform('soma', 0.120), "e_rev": 50.0}

In [50]:
na["conductance_density"].value_in(nu_morph, 0)

AttributeError: 'NeuroMLMorphology' object has no attribute '_soma_index'

In [51]:
if isinstance(nu_morph._morphology, neuroml.Morphology):
    print("yes")

yes


In [52]:
sections = {}
unresolved_connections = []
for index, segment in enumerate(nu_morph.segments):
    print(index, segment)

0 <Segment|0|soma>
1 <Segment|1|dendrites>


In [53]:
from neuron import nrn

In [54]:
PROXIMAL = 0
DISTAL = 0

In [55]:
sections = {}
section_labels = {}
unresolved_connections = []
for index, segment in enumerate(nu_morph.segments):
    section = nrn.Section(name=segment.name)
    section.L = segment.length
    section(PROXIMAL).diam = segment.proximal.diameter
    section(DISTAL).diam = segment.distal.diameter
    section.nseg = 1
    segment_id = segment.id
    if segment.parent:
        parent_id = segment.parent.id
        connection_point = DISTAL
        if segment.parent.id in sections:
            section.connect(sections[parent_id], connection_point, PROXIMAL)
        else:
            unresolved_connections.append((segment_id, parent_id))
    sections[segment_id] = section
    if segment.name == "soma":
        nu_morph._soma_index = segment_id
    if segment.name is not None:
        section_labels[segment.name] = section
    segment._section = section
for section_id, parent_id in unresolved_connections:
    sections[section_id].connect(sections[parent_id], DISTAL, PROXIMAL)

In [56]:
for index, segment in enumerate(nu_morph.segments):
    print(index, segment)

0 <Segment|0|soma>
1 <Segment|1|dendrites>


In [57]:
na["conductance_density"].value_in(nu_morph, 0)

0.12

In [58]:
na["conductance_density"].value_in(nu_morph, 1)

0.0

In [59]:
for index, id in enumerate(sections):
    print(index, id)

0 0
1 1


In [60]:
section(0) # argument is id not index

dendrites(0)

In [61]:
kdr={"conductance_density": by_distance(apical_dendrites(), lambda d: 0.05*d/200.0), "e_rev": -77.0}

In [62]:
kdr["conductance_density"]

In [63]:
from pyNN.morphology import MorphologyFilter

In [64]:
kdr["conductance_density"].selector

In [65]:
if isinstance(kdr["conductance_density"].selector, MorphologyFilter):
    print("yes")

yes


In [66]:
kdr["conductance_density"].selector(nu_morph)

Exception: No neurites labelled as apical dendrite

In [67]:
kdr={"conductance_density": by_distance(dendrites(), lambda d: 0.05*d/200.0), "e_rev": -77.0}

In [68]:
kdr["conductance_density"].selector(nu_morph)

Exception: No neurites labelled as dendrites

In [35]:
from pyNN.morphology import basal_dendrites

In [36]:
kdr={"conductance_density": by_distance(basal_dendrites(), lambda d: 0.05*d/200.0), "e_rev": -77.0}

In [37]:
kdr["conductance_density"].selector(nu_morph)

Exception: No neurites labelled as basal dendrite

In [70]:
nu_morph.section_groups

{}